# 07 - LLMs (Zero-Shot Classification)

**Notebook de referência para classificação zero-shot com LLMs**

Este notebook demonstra como usar modelos de linguagem grandes para classificação sem fine-tuning:
- Qwen3 (1.7B, 4B)
- Gemma 3 (4B)
- Llama 3.2 (3B)

---

## Configuração Kaggle

| Modelo | Kaggle Input | VRAM |
|--------|--------------|------|
| Qwen3 1.7B | `QwenLM/Qwen3` → `1.7B` | ~4GB |
| Qwen3 4B | `QwenLM/Qwen3` → `4B` | ~9GB |
| Gemma 3 4B | `google/gemma-3` → `4b` | ~9GB |
| Llama 3.2 3B | `meta-llama/Llama-3.2` → `3B` | ~7GB |

**Settings:**
- Internet → **OFF**
- Accelerator → **GPU T4 x2** ou **P100**
- Add Data → Models → escolher modelo acima

**Atenção:** LLMs funcionam offline mas precisam do modelo como Input!

In [ ]:
# =============================================================================
# SETUP E IMPORTS
# =============================================================================
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - LLM Zero-Shot Classification")
print("="*60)

# =============================================================================
# CONFIGURAÇÕES
# =============================================================================
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# Escolher modelo (descomentar o desejado)
# =========================================
MODEL_NAME = 'qwen3-1.7b'  # <-- ALTERAR AQUI

MODEL_CONFIGS = {
    'qwen3-1.7b': {
        'paths': [
            '/kaggle/input/qwen3/transformers/1.7b/1',
            '/kaggle/input/qwen3/transformers/1.7b',
        ],
        'description': 'Qwen3 1.7B - compacto e eficiente'
    },
    'qwen3-4b': {
        'paths': [
            '/kaggle/input/qwen3/transformers/4b/1',
            '/kaggle/input/qwen3/transformers/4b',
        ],
        'description': 'Qwen3 4B - mais capacidade'
    },
    'gemma3-4b': {
        'paths': [
            '/kaggle/input/gemma-3/transformers/4b/1',
            '/kaggle/input/gemma-3/transformers/4b',
        ],
        'description': 'Gemma 3 4B - Google'
    },
    'llama3-3b': {
        'paths': [
            '/kaggle/input/llama-3.2/transformers/3b/1',
            '/kaggle/input/llama-3.2/transformers/3b',
        ],
        'description': 'Llama 3.2 3B - Meta'
    },
}

# Encontrar path do modelo
config = MODEL_CONFIGS[MODEL_NAME]
MODEL_PATH = None

for path in config['paths']:
    if os.path.exists(path):
        MODEL_PATH = path
        break

if MODEL_PATH is None:
    print(f"\n⚠️ Modelo '{MODEL_NAME}' não encontrado!")
    print("Datasets disponíveis:")
    for item in os.listdir('/kaggle/input'):
        print(f"  - {item}")
    raise FileNotFoundError("Adicione o modelo ao notebook!")

USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

print(f"✓ Modelo: {MODEL_NAME}")
print(f"✓ Path: {MODEL_PATH}")
print(f"✓ {config['description']}")
print(f"✓ GPU: {USE_GPU}")
print(f"✓ Device: {device}")

In [ ]:
# =============================================================================
# CARREGAR DADOS
# =============================================================================
print("\n[1/4] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f"Train: {train.shape}")
print(f"Test: {test.shape}")

print("\nDistribuição das classes no treino:")
print(train['target'].value_counts().sort_index())

In [ ]:
# =============================================================================
# CARREGAR MODELO
# =============================================================================
print("\n[2/4] Carregando modelo...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if USE_GPU else torch.float32,
    device_map='auto' if USE_GPU else None,
    trust_remote_code=True,
)

if not USE_GPU:
    model = model.to(device)

model.eval()
print(f"✓ Modelo carregado!")
print(f"✓ Parâmetros: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# =============================================================================
# PROMPT TEMPLATE
# =============================================================================

SYSTEM_PROMPT = """Você é um especialista em radiologia mamária. 
Sua tarefa é classificar laudos de mamografia na escala BI-RADS (0-6).

Categorias BI-RADS:
- 0: Avaliação incompleta
- 1: Negativo (normal)
- 2: Achado benigno
- 3: Provavelmente benigno
- 4: Suspeita de malignidade
- 5: Altamente sugestivo de malignidade
- 6: Malignidade conhecida

Responda APENAS com um número de 0 a 6."""

def create_prompt(report):
    return f"""<|system|>
{SYSTEM_PROMPT}
<|user|>
Classifique o seguinte laudo de mamografia:

{report}

Categoria BI-RADS (0-6):
<|assistant|>
"""

# Testar prompt
sample_report = train['report'].iloc[0]
print("Exemplo de prompt:")
print(create_prompt(sample_report[:200] + "..."))

In [ ]:
# =============================================================================
# FUNÇÃO DE CLASSIFICAÇÃO
# =============================================================================
import re

def classify_report(report, model, tokenizer, max_new_tokens=10):
    """Classifica um laudo usando o LLM."""
    prompt = create_prompt(report)
    
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=2048)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número da resposta
    numbers = re.findall(r'[0-6]', response)
    if numbers:
        return int(numbers[0])
    
    # Fallback: classe mais comum (2)
    return 2

# Testar
sample_pred = classify_report(train['report'].iloc[0], model, tokenizer)
print(f"Predição de teste: {sample_pred}")
print(f"Label real: {train['target'].iloc[0]}")

In [ ]:
# =============================================================================
# INFERÊNCIA NO TESTE
# =============================================================================
print("\n[3/4] Classificando...")

predictions = []

for idx, row in tqdm(test.iterrows(), total=len(test), desc="Classificando"):
    pred = classify_report(row['report'], model, tokenizer)
    predictions.append(pred)

print(f"\n✓ {len(predictions)} predições geradas")

In [ ]:
# =============================================================================
# SUBMISSÃO
# =============================================================================
print("\n[4/4] Criando submissão...")

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print(f"\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())

---

## Técnicas Avançadas

### Few-Shot Learning

Adicionar exemplos no prompt para melhorar a classificação:

```python
FEW_SHOT_EXAMPLES = """
Exemplo 1:
Laudo: "Mamas simétricas, sem nódulos ou calcificações suspeitas."
Classificação: 1

Exemplo 2:
Laudo: "Nódulo sólido de contornos irregulares, medindo 2cm."
Classificação: 4
"""
```

### Chain-of-Thought

Pedir ao modelo que explique o raciocínio antes de classificar:

```python
PROMPT = """
Analise o laudo passo a passo:
1. Identifique os achados principais
2. Avalie a suspeição de malignidade
3. Classifique na escala BI-RADS (0-6)

Laudo: {report}

Análise:
"""
```

### Batch Processing

Para acelerar a inferência, processar múltiplos laudos por vez (se houver VRAM suficiente).

---

## Validação no Treino (Opcional)

Antes de submeter, validar a performance no conjunto de treino.

In [ ]:
# =============================================================================
# VALIDAÇÃO (OPCIONAL)
# =============================================================================
# Descomentar para validar no treino

'''
from sklearn.metrics import f1_score, classification_report

# Usar uma amostra para validação rápida
n_samples = 100
sample = train.sample(n=n_samples, random_state=SEED)

val_preds = []
for idx, row in tqdm(sample.iterrows(), total=len(sample), desc="Validando"):
    pred = classify_report(row['report'], model, tokenizer)
    val_preds.append(pred)

f1 = f1_score(sample['target'], val_preds, average='macro')
print(f"\nF1-Macro (amostra de {n_samples}): {f1:.4f}")
print("\nClassification Report:")
print(classification_report(sample['target'], val_preds))
'''